# Chapter 05: 스킬 선택 (Skill Selection)

이 노트북에서는 LangGraph를 사용한 조건부 라우팅과 스킬 선택을 학습합니다.

## 주요 내용
- LangGraph 조건부 엣지
- 문제 분류 및 라우팅
- 계층적 워크플로우

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-repo/notebook/ch05_skill_selection.ipynb)


In [ ]:
!pip install -q langchain langchain-openai langgraph python-dotenv


In [ ]:
import os
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except:
    pass
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = "sk-your-api-key-here"


## LangGraph 조건부 라우팅 예제

고객 지원 요청을 billing 또는 technical로 분류하고 적절한 핸들러로 라우팅합니다.


In [ ]:
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage

# LLM 초기화
llm = init_chat_model(model="gpt-5-mini")

# State 정의
class AgentState(TypedDict):
    user_message: str
    user_id: str
    issue_type: Optional[str]
    step_result: Optional[str]
    response: Optional[str]

print("✅ State 및 LLM 설정 완료")


In [ ]:
# 노드 정의
def categorize_issue(state: AgentState) -> AgentState:
    """문제를 billing 또는 technical로 분류"""
    prompt = f"이 지원 요청을 'billing' 또는 'technical'로 분류하세요.\n\n메시지: {state['user_message']}"
    response = llm.invoke([HumanMessage(content=prompt)])
    kind = response.content.strip().lower()
    if "billing" in kind:
        kind = "billing"
    else:
        kind = "technical"
    return {"issue_type": kind}

def handle_invoice(state: AgentState) -> AgentState:
    return {"step_result": f"Invoice details for {state['user_id']}"}

def handle_refund(state: AgentState) -> AgentState:
    return {"step_result": "Refund process initiated"}

def handle_login(state: AgentState) -> AgentState:
    return {"step_result": "Password reset link sent"}

def summarize_response(state: AgentState) -> AgentState:
    details = state.get("step_result", "")
    prompt = f"다음 내용을 바탕으로 간결한 고객 응답을 작성하세요: {details}"
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"response": response.content.strip()}

print("✅ 노드 함수 정의 완료")


In [ ]:
# 그래프 구성
graph_builder = StateGraph(AgentState)

# 노드 추가
graph_builder.add_node("categorize_issue", categorize_issue)
graph_builder.add_node("handle_invoice", handle_invoice)
graph_builder.add_node("handle_refund", handle_refund)
graph_builder.add_node("handle_login", handle_login)
graph_builder.add_node("summarize_response", summarize_response)

# 엣지 추가
graph_builder.add_edge(START, "categorize_issue")

# 조건부 라우팅
def top_router(state: AgentState):
    return "billing" if state["issue_type"] == "billing" else "technical"

graph_builder.add_conditional_edges(
    "categorize_issue",
    top_router,
    {"billing": "handle_invoice", "technical": "handle_login"}
)

# 후속 처리
graph_builder.add_edge("handle_invoice", "summarize_response")
graph_builder.add_edge("handle_login", "summarize_response")

# 컴파일
graph = graph_builder.compile()
print("✅ 그래프 컴파일 완료")


In [ ]:
# 테스트 실행
initial_state = {
    "user_message": "안녕하세요, 인보이스 관련 도움을 받고 싶습니다.",
    "user_id": "U1234"
}

result = graph.invoke(initial_state)

print("=" * 50)
print(f"🏷️ 분류된 문제 유형: {result['issue_type']}")
print(f"📝 처리 결과: {result['step_result']}")
print(f"🤖 최종 응답: {result['response']}")
